In [1]:
from time import time
from datetime import datetime
import numpy as np
seq = 12
dt = 8  
def multiply(A,B):
    return [a*b for a,b in zip(A,B)]
def divide(A,B):
    return [a/b for a,b in zip(A,B)]

def minus(A,B):
    return [a-b for a,b in zip(A,B)]
def log(txt):
    f= open("log%i.txt"%seq,"a+")
    f.write('%s: %s \r\n'%(datetime.now().strftime('{%Y-%m-%d %H:%M:%S}'),txt))
    print(txt)
    f.close() 
def initial_log():
    f= open("log%i.txt"%seq,"w+")
    f.write('%s: starting.... \r\n'%(datetime.now().strftime('{%Y-%m-%d %H:%M:%S}')))
    f.close() 
import pickle

def savedf():
    with open('df%i.dat'%dt,'wb') as f:
        pickle.dump((X_train, X_test, y_train, y_test),f)

def loaddf():
    global X_train, X_test, y_train, y_test
    with open('df%i.dat'%dt,'rb') as f:
        X_train, X_test, y_train, y_test = pickle.load(f)
    print("Shape of X_train:", len(X_train),len(X_train[0]))
    print("Shape of X_test:",len(X_test),len(X_test[0]))

      
def save():
    with open('data%i.dat'%seq,'wb') as f:
        pickle.dump((i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse),f)
def load():
    global i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse
    with open('data%i.dat'%seq,'rb') as f:
        i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse = pickle.load(f)

def load1():
    global i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse
    with open('data4.dat','rb') as f:
        i,mdl_1,mdl_2,mdl_3,RMSETest,RMSETrain,last_depth,last_rmse = pickle.load(f)
import os
os.environ["PATH"] += os.pathsep +"C:\\Users\\User\\Downloads\\graphviz-2.38\\release\\bin"


In [2]:
from numpy import var

class Splitter:
    def __init__(self,X,Y):
        self.X = X
        self.Y = Y

    def split(self,index,value):
        """ 
        Split X and Y based on the given variable index and value
  
        Parameters: 
            index (int): index of variable to splitted on
            value (float): the value of splitting varaible
          
        Returns: 
            right_x,left_x,rigth_y,left_y
        """
        right_x = [x for x in self.X if x[index]>=value]
        left_x = [x for x in self.X if x[index]<value]
        
        right_y = [self.Y[i] for i,x in enumerate(self.X) if x[index]>=value]
        left_y = [self.Y[i] for i,x in enumerate(self.X) if x[index]<value]
        return right_x,left_x,right_y,left_y
    
    def weighted_var(self,Y):
        if len(Y) ==0:
            return float('inf')
        return var(Y)/len(Y)
    
    def SE(self,Y):
        if len(Y) ==0: return float('inf')
        mean = sum(Y)/len(Y)
        SE = sum([(y-mean)**2 for y in Y])
        return SE
        
    
    def splitting_cost(self,right_x,left_x,right_y,left_y):
        """ 
        Compute cost of spliting.
        in the current version, we have used varaince based cost function
  
        Parameters: 
            right_x,left_x,rigth_y,left_y
          
        Returns: 
            float: the cost of computing
        """        
        return (self.SE(right_y)+self.SE(left_y))/(len(right_y)+len(left_y))
    
    def possibleValues(self,index):
        values = [x[index] for x in self.X]
        values.sort()
        max_values = 100
        if len(values)<max_values:
            return values
        
        return set([np.percentile(values,i*100//max_values) for i in range(max_values)])
    
    def test_splitting(self,index,value):
        right_x,left_x,right_y,left_y = self.split(index,value)
        return  self.splitting_cost(right_x,left_x,right_y,left_y),len(right_y),len(left_y)
    
    def find_best_splitting(self,min_leaf_treshold=0,min_split=0):
        """ 
        Find best splitting and return index and value of splitting variable respectively
       
        Returns: 
            int,float: the index and value of best possible splitting variables
        """ 
        
        best_splitting = (self.SE(self.Y)/len(self.Y))
        best_splitting_index = -1
        best_splitting_value = 0
        if len(self.Y)<min_split:
            return best_splitting_index,best_splitting_value            
        for index in range(len(self.X[0])):
            for value in self.possibleValues(index):
                splitting_cost,left_cnt,right_cnt = self.test_splitting(index,value)
#               print('splitting at %f: cost->%f'%(value,splitting_cost))
                if splitting_cost < best_splitting and left_cnt >= min_leaf_treshold and right_cnt >= min_leaf_treshold:
                    best_splitting = splitting_cost
                    best_splitting_index = index
                    best_splitting_value = value
        return best_splitting_index,best_splitting_value


In [3]:
id = 0
last_leaf_no = 1
j = 0
def create_Tree():
    global last_leaf_no
    last_leaf_no = 1
    return Node(0)
class Node:
    isLeaf = True
    Left = None
    Right = None
    Value = None
    Depth = 0
    splitting_var_index = 0
    splitting_value = 0
    cnt = 0
    
    
    def __init__(self,depth):
        global id
        id+=1
        self.ID = id
        self.isLeaf = True
        self.Depth = depth
    def predict(self,X):
        return [self.predict_one(x).Value for x in X]
        
    def predict_one(self,X):
        if type(X) != list:
            raise TypeError("predict_one.X: expected list but recieved %s (%s)"%(X,type(X)))
        try:
            if self.isLeaf:
                self.cnt +=1
                return self
            if X[self.splitting_var_index] >= self.splitting_value:
                return self.Right.predict_one(X)
            else:
                return self.Left.predict_one(X)
        except IndexError:
            print('index: %i'%self.splitting_var_index)
            raise
    def get_leafs(self):
        if self.isLeaf:
            return [self]
        return self.Right.get_leafs()+self.Left.get_leafs()   
    
    def fit(self,X,Y,max_depth):
        global last_leaf_no

        #tree reached maximum depth
        if max_depth <= self.Depth:
            self.isLeaf = True
            self.leafID = last_leaf_no
            last_leaf_no+=1
            self.fit_leaf_value(Y)
            return 
        
        #continue splitting if more depth allowed
        splitter = Splitter(X,Y)
        best_splitting_index,best_splitting_value = splitter.find_best_splitting()
        if best_splitting_index == -1:
            #better to not split
            self.isLeaf = True
            self.leafID = last_leaf_no
            last_leaf_no+=1
            self.fit_leaf_value(Y)
        else:
            right_x,left_x,right_y,left_y = splitter.split(best_splitting_index,best_splitting_value)
            self.isLeaf = False
            self.splitting_var_index = best_splitting_index
            self.splitting_value = best_splitting_value
            self.Right = Node(self.Depth+1)
            self.Left = Node(self.Depth+1)
            self.Right.fit(right_x,right_y,max_depth)
            self.Left.fit(left_x,left_y,max_depth)
    def fit_leaf_value(self,Y):
        self.Value = sum(Y)/len(Y)

        
    def tune(self,X,R,T):
        #self.X = X
       # self.Y = ['%f:%f'%(r,t) for r,t in zip(R,T)]
        if len(X) != len(R):
            raise TypeError("tune function: X and R should be with same size. recieved <%i>.<%i>"%(len(X),len(R)))
        if len(X) != len(T):
            raise TypeError("tune function: X and T should be with same size. recieved <%i>.<%i>"%(len(X),len(T)))
            
        if self.isLeaf:
            if sum(multiply(T,T)) == 0:
                self.value = 1
            else:
                self.Value = sum(multiply(R,T))/sum(multiply(T,T))
            return
        
        splitter = Splitter(X,R)
        right_x,left_X,right_R,left_R = splitter.split(self.splitting_var_index,self.splitting_value)
        splitter = Splitter(X,T)
        right_x,left_X,right_T,left_T = splitter.split(self.splitting_var_index,self.splitting_value)
        self.Right.tune(right_x,right_R,right_T)
        self.Left.tune(left_X,left_R,left_T)
        
    def draw_tree(self):
        global j
        from graphviz import Source
        from io import BytesIO
        import time
        timestr = time.strftime("%Y%m%d-%H%M%S")
        from matplotlib import pyplot as plt
        from matplotlib import image
        g = Source('digraph {\n\tgraph [rankdir=UT]\n'+self.print_tree()+'}')
        g.render('round-table%i.gv'%j, view=True)
        j+=1
        #return img
        
    def get_leafcounts(self):
        if self.isLeaf:
            return 1
        return self.Left.get_leafcounts()+self.Right.get_leafcounts()
    
    def print_tree(self,prefix=''):
        '\t0 [label="f0<8.40779079e-45] yes=1,no=2,missing=1\n\t1:[f0<2.80259693e-45] yes=3,no=4,missing=3\n\t\t3:leaf=7.46349096\n\t\t4:leaf=-0.278686523\n\t2:[f0<8.40779079e-45] yes=5,no=6,missing=5\n\t\t5:leaf=-0\n\t\t6:leaf=-0.188493282\n'
        if not self.isLeaf:
            ret = '\t%i [label="f%i>=%f"]\n\t%i -> %i [label="yes" color="#0000FF"]\n\t %i -> %i [label=no color="#FF0000"]\n'%(
                self.ID,self.splitting_var_index,self.splitting_value,self.ID,self.Right.ID,self.ID,self.Left.ID)
            ret += self.Right.print_tree(prefix+'\t')
            ret += self.Left.print_tree(prefix+'\t')
            return ret
        else:
            return '\t%i [label="leaf=%f\nCount=%i"]\n'%(self.ID,self.Value,self.cnt)
    def get_leaf_values(self):
        if self.isLeaf:
            return [self.Value]
        return self.Right.get_leaf_values()+self.Left.get_leaf_values()
    
'''digraph {
	graph [rankdir=UT]
	0 [label="f0<8.40779079e-45"]
	0 -> 1 [label="yes, missing" color="#0000FF"]
	0 -> 2 [label=no color="#FF0000"]
	1 [label="f0<2.80259693e-45"]
	1 -> 3 [label="yes, missing" color="#0000FF"]
	1 -> 4 [label=no color="#FF0000"]
	3 [label="leaf=7.46349096"]
	4 [label="leaf=-0.278686523"]
	2 [label="f0<8.40779079e-45"]
	2 -> 5 [label="yes, missing" color="#0000FF"]
	2 -> 6 [label=no color="#FF0000"]
	5 [label="leaf=-0"]
	6 [label="leaf=-0.188493282"]
}'''       
        

'digraph {\n\tgraph [rankdir=UT]\n\t0 [label="f0<8.40779079e-45"]\n\t0 -> 1 [label="yes, missing" color="#0000FF"]\n\t0 -> 2 [label=no color="#FF0000"]\n\t1 [label="f0<2.80259693e-45"]\n\t1 -> 3 [label="yes, missing" color="#0000FF"]\n\t1 -> 4 [label=no color="#FF0000"]\n\t3 [label="leaf=7.46349096"]\n\t4 [label="leaf=-0.278686523"]\n\t2 [label="f0<8.40779079e-45"]\n\t2 -> 5 [label="yes, missing" color="#0000FF"]\n\t2 -> 6 [label=no color="#FF0000"]\n\t5 [label="leaf=-0"]\n\t6 [label="leaf=-0.188493282"]\n}'

In [4]:
    from copy import deepcopy
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    from scipy.sparse import csr_matrix
    from scipy import sparse
    import numpy as np
    from scipy.optimize import lsq_linear
    from sklearn.model_selection import train_test_split

    class FractBoosting:

        def __init__(self,learning_rate):
            self.learning_rate = learning_rate
            self.trees = []

        def predict(self,X,ignore_last=0):
            Y = []
            tree_filtered = self.trees[:max(1,len(self.trees)-ignore_last)]
            print('prediciting using %i trees'%len(tree_filtered))
            for x in X:
                predictions = [S.predict([x])[0]*T.predict([x])[0] for S,T in tree_filtered]
                predict = predictions[0] + self.learning_rate*sum(predictions[1:])
                Y.append(predict)

            return Y

        def fit(self,X,Y,max_depth,max_trees):
            """ 
            Fit the boosting trees on the given data

            Parameters: 
                X (list of list): the input X
                Y (list): input Y
                max_depth (int): maximum depth of trees
                max_trees (int): the number of trees
            """ 

            lastRMSE = self.rmse(X,Y)
            improvment = []
            for i in range(1,max_trees):
                t0 = time()
                depth = int(1+max_depth*sqrt(i/max_trees))
                log('starting itteration %i with depth %i'%(i,depth))
    #            print('itteration %i, begining, mean loss: %f, tree depth: %i'%(i,self.rmse(X,Y),depth))
                self.fit_one_step(X,Y,depth)

                current_rmse = self.rmse(X,Y)
                RMSEDif = lastRMSE - current_rmse
                lastRMSE = current_rmse
                cost = T.get_leafcounts()
                imp = cost/RMSEDif
                improvment.append(imp)
                log('itteration %i done in %0.3fs, RMSE: %f, Cost/improvement: %f'%(i,time() - t0,current_rmse,imp))
            return improvment
        def create_root_tree(self,Y):
            root_s = create_Tree()
            root_s.Value = 1
            root_t = create_Tree()
            root_t.fit_leaf_value(Y)
            self.trees = [(root_s,root_t)]

        def fit_one_step(self,X_TRAIN,Y_TRAIN,depth,backprop_level):
            X, X1, Y, Y1 = train_test_split(X_TRAIN, Y_TRAIN, test_size=0.2,shuffle=False)
            Y_r = self.get_residual(X,Y)
            T = create_Tree()
            T.fit(X,Y_r,depth)
            Y_t = T.predict(X)
            S = deepcopy(self.trees[-1][1])
            S.tune(X,Y_r,Y_t)
            self.trees.append((S,T))
            X, _, Y, _ = train_test_split(X_TRAIN, Y_TRAIN, test_size=0.8,shuffle=False)
            X = X+X1
            Y = Y+Y1
            self.back_prop(X,Y,backprop_level)
            
        def back_prop(self,X,Y,level):
            global A4
            print(len(X),len(Y))
            #self.trees[-1][0].draw_tree()
            h = self.predict(X,level)
            R = np.array(minus(Y , h))
            tr = self.trees[max(1,len(self.trees)-level):]
            print('computing last %i trees of S [total trees: %i]'%(len(tr),len(self.trees)))
            i = 0
            S_nodes =[]
            s_values = []
            for s,_ in tr:
                for leaf in s.get_leafs():
                    leaf.leaf_index = i
                    i+=1
                    S_nodes.append(leaf)
                    s_values.append(leaf.Value)
            row = []
            col = []
            data =[]
            for i,x in enumerate(X):
                for s,t in tr:
                    s_leaf = s.predict_one(x)
                    t_leaf = t.predict_one(x)
                    row.append(i)
                    col.append(s_leaf.leaf_index)
                    data.append(t_leaf.Value)
            A = sparse.csr_matrix((data, (row, col)), shape=(len(X), len(S_nodes))) 
            ##U = A.transpose().dot(A).todense()
            #V = A.transpose().dot(R)
            #A4 = np.linalg.pinv(U,hermitian =True).dot(V)
            #print(A4.shape)
            A4 = lsq_linear(A,R, bounds=(-0.01, .1)).x
            print("S Values before ",s_values)
            print('S Values after ',A4)
            #A4 = A4.todesne()
#            A1 = A.transpose().dot(A)
#            print(np.linalg.matrix_rank(A.todense()),np.linalg.matrix_rank(A1.todense()))
#            A2 = sparse.csr_matrix(inv(A1))
#            A3 = A.transpose().dot(R)
#            A4 = (A2.dot(A3)).todense()
            print('rmse before:',self.rmse(X,Y))
            r1 = self.get_residual(X,Y)
            for i,s1 in enumerate(S_nodes):
                s1.Value = A4[i]
            #self.trees[-1][0].draw_tree()
            print('rmse after:',self.rmse(X,Y))
            r2 = self.get_residual(X,Y)
            #print('Residual before ',r1)
            #print('Residual after ',r2)
            #print('diff ',minus(r1,r2))
            
        def get_residual(self,X,Y):
            h = self.predict(X)
            Y_r = minus(Y , h)
            return Y_r
        def print_trees(self):
            for i,(s,t) in enumerate(self.trees):
                print('%ith fractal tree\n'%i)
                print('-----------------\n')
                print('S:\n')
                s.print_tree()
                print('-----------------\n')
                print('T:\n')
                t.print_tree()
                print('\n-----------------\n')

        def rmse(self,X,Y):
            h = self.predict(X)
    #        R = minus(Y , h)
    #        print('Residual:',R)
            return sqrt(mean_squared_error(h, Y))


In [ ]:
from numpy import std
import xgboost as xgb
from threading import Thread
from xgboost import plot_tree
from sklearn.metrics import mean_squared_error
max_depth=5

loaddf()
#initial_log()
mdl_1 = FractBoosting(1)
mdl_2 = FractBoosting(1)
mdl_3 = FractBoosting(1)
mdl_1.create_root_tree(y_train)
mdl_2.create_root_tree(y_train)
mdl_3.create_root_tree(y_train)
last_rmse = mdl_3.rmse(X_train,y_train)
last_depth = 1

m = sum(y_train)/len(y_train)
y_train_xgb = [y-m for y in y_train]

RMSETest = [[],[],[],[],[],[]]
RMSETrain =  [[],[],[],[],[],[]]
i=0
try:
    load()
    i
except Exception as e:
    print(e)
    initial_log()
try:
    while i<200:
        processes = []




        ################Fixed depth
        mdl_1.fit_one_step(X_train,y_train,max_depth,1)
        RMSETrain[0].append( mdl_1.rmse(X_train,y_train))
        RMSETest[0].append( mdl_1.rmse(X_test,y_test))    
        print('1 Back')


        
        ###############Linear depth
        #depth = min(5,int(1+max_depth*(i/50)))
        mdl_2.fit_one_step(X_train,y_train,max_depth,10)  
        RMSETrain[1].append( mdl_2.rmse(X_train,y_train))
        RMSETest[1].append( mdl_2.rmse(X_test,y_test))    
        print('2 Back')


        ###############dynamic depth
        mdl_3.fit_one_step(X_train,y_train,max_depth,20)
        #mdl3_rmse = mdl_3.rmse(X_test,y_test)
        RMSETrain[2].append( mdl_3.rmse(X_train,y_train))
        RMSETest[2].append( mdl_3.rmse(X_test,y_test))
        print('3 Back')
        


        ##################XGBoost
        xg_reg = xgb.XGBRegressor( learning_rate = 0.1,
                        max_depth = 5,  n_estimators = i)
        xg_reg.fit(X_train,y_train_xgb)    
        preds = xg_reg.predict(X_train)
        RMSETrain[3].append(np.sqrt(mean_squared_error(y_train, [p+m for p in preds])))
        preds = xg_reg.predict(X_test)
        RMSETest[3].append(np.sqrt(mean_squared_error(y_test, [p+m for p in preds])))
        print('XGB')
        i+=1
        
        save()
                           
        log('\r\n-----------------------------------------------\r\n'+
            'ittr %i  \tTrain\t\tTest\t\tDiff\t\tRVar\tSVar\tTVar\r\n'%i+
            'FGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t%.3f\t\t%.3f\r\n'%(1,RMSETrain[0][-1],RMSETest[0][-1],RMSETest[0][-1]-RMSETrain[0][-1],std(mdl_1.get_residual(X_train,y_train)),std(mdl_1.trees[-1][0].get_leaf_values()),std(mdl_1.trees[-1][1].get_leaf_values()))+
            'FGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t%.3f\t\t%.3f\r\n'%(10,RMSETrain[1][-1],RMSETest[1][-1],RMSETest[1][-1]-RMSETrain[1][-1],std(mdl_2.get_residual(X_train,y_train)),std(mdl_2.trees[-1][0].get_leaf_values()),std(mdl_2.trees[-1][1].get_leaf_values()))+
            'FGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f\t\t%.3f\t%.3f\t\t%.3f\r\n'%(20,RMSETrain[2][-1],RMSETest[2][-1],RMSETest[2][-1]-RMSETrain[2][-1],std(mdl_3.get_residual(X_train,y_train)),std(mdl_3.trees[-1][0].get_leaf_values()),std(mdl_3.trees[-1][1].get_leaf_values()))+
            'XGB[%i]:  \t%.3f\t\t%.3f\t\t%.3f \r\n'%(max_depth,RMSETrain[3][-1],RMSETest[3][-1],RMSETest[3][-1]-RMSETrain[3][-1]))


except Exception as e:
    log(e)
    raise


Shape of X_train: 22865 9
Shape of X_test: 22865 9
prediciting using 1 trees
[Errno 2] No such file or directory: 'data12.dat'
prediciting using 1 trees
9146 9146
prediciting using 1 trees
computing last 1 trees of S [total trees: 2]
S Values before  [1.0000000000000377]
S Values after  [0.1]
prediciting using 2 trees
rmse before: 5.161011335419127
prediciting using 2 trees
prediciting using 2 trees
rmse after: 5.972890061678466
prediciting using 2 trees
prediciting using 2 trees
prediciting using 2 trees
1 Back
prediciting using 1 trees
9146 9146
prediciting using 1 trees
computing last 1 trees of S [total trees: 2]
S Values before  [1.0000000000000377]
S Values after  [0.1]
prediciting using 2 trees
rmse before: 5.161011335419127
prediciting using 2 trees
prediciting using 2 trees
rmse after: 5.972890061678466
prediciting using 2 trees
prediciting using 2 trees
prediciting using 2 trees
2 Back
prediciting using 1 trees
9146 9146
prediciting using 1 trees
computing last 1 trees of S [

prediciting using 4 trees
rmse after: 5.688003920095772
prediciting using 4 trees
prediciting using 4 trees
prediciting using 4 trees
2 Back
prediciting using 3 trees
9146 9146
prediciting using 1 trees
computing last 3 trees of S [total trees: 4]
S Values before  [0.09999999996573142, 0.09999999994477589, 0.09999999994784292, 0.09999999999851505, 0.09999999999867487, 0.09999999999795639, 0.09999999997249266, 0.09999999999801472, 0.0999999999975446, 0.09000000000000001, 0.09999999995141116, -0.009999993311526541, 0.09999999999999999, 0.0999999999797296, 0.09999999994811547, 0.09999999999999999, 0.03506602972458838, 0.0999999999980723, 0.09999999991543476, 0.09999999995249066, 0.09999999996943923, 0.09999999994684224, 0.09999999999760846, 0.09999999999508528, 0.09999999999999999, -0.009999998475254835, 0.09999999996728318, 0.09999999999510435, 0.09999999999885612, 0.09999999994999247, 0.09999999994907088, 0.09999999999819623, 0.09999999999771844, 0.7304288284229679, 1.0927131371379215, 

rmse before: 5.06959655745747
prediciting using 5 trees
prediciting using 5 trees
rmse after: 5.574865147153659
prediciting using 5 trees
prediciting using 5 trees
prediciting using 5 trees
3 Back
[05:33:09] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 5 trees
prediciting using 5 trees
prediciting using 5 trees

-----------------------------------------------
ittr 4  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	5.544		5.577		0.034		5.544	0.027		3.060
FGB[10]:  	5.544		5.577		0.034		5.544	0.027		3.060
FGB[20]:  	5.544		5.577		0.034		5.544	0.027		3.060
XGB[5]:  	5.662		5.691		0.029 

prediciting using 5 trees
9146 9146
prediciting using 5 trees
computing last 1 trees of S [total trees: 6]
S Values before  [1.1834259631772406, 0.1339629930508796, -0.22947937955371717, 1.2347036500465334, 1.0512976117618062, 1.079948809512429, 0.9200901146773292, 1.0503392786461

rmse before: 4.948914040361371
prediciting using 6 trees
prediciting using 6 trees
rmse after: 5.46739868733313
prediciting using 6 trees
prediciting using 6 trees
prediciting using 6 trees
3 Back
[05:38:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 6 trees
prediciting using 6 trees
prediciting using 6 trees

-----------------------------------------------
ittr 5  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	5.437		5.478		0.041		5.437	0.036		3.054
FGB[10]:  	5.437		5.478		0.041		5.437	0.036		3.054
FGB[20]:  	5.437		5.478		0.041		5.437	0.036		3.054
XGB[5]:  	5.544		5.581		0.036 

prediciting using 6 trees
9146 9146
prediciting using 6 trees
computing last 1 trees of S [total trees: 7]
S Values before  [0.6785930416262337, 1.1092894565792546, 1.0047682876213282, 1.2057307441839404, -0.7512376595699077, 0.4292570426446427, 0.9053436964881171, 1.0868586347933

rmse before: 4.951381074301865
prediciting using 7 trees
prediciting using 7 trees
rmse after: 5.383874986998493
prediciting using 7 trees
prediciting using 7 trees
prediciting using 7 trees
3 Back
[05:43:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 7 trees
prediciting using 7 trees
prediciting using 7 trees

-----------------------------------------------
ittr 6  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	5.353		5.404		0.050		5.353	0.040		3.019
FGB[10]:  	5.354		5.404		0.050		5.354	0.040		3.019
FGB[20]:  	5.354		5.404		0.050		5.354	0.040		3.019
XGB[5]:  	5.441		5.485		0.044 

prediciting using 7 trees
9146 9146
prediciting using 7 trees
computing last 1 trees of S [total trees: 8]
S Values before  [1.3433417025482015, 1.2116945209144832, 1.066378064698791, 1.2702535383650182, 0.42732236910678906, 1.2835340964386086, 1.7540152956955979, 0.1325617647692

...

rmse before: 7.607897878520926
prediciting using 83 trees
prediciting using 83 trees
rmse after: 4.098603362331054
prediciting using 83 trees
prediciting using 83 trees
prediciting using 83 trees
3 Back
[14:20:48] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 83 trees
prediciting using 83 trees
prediciting using 83 trees

-----------------------------------------------
ittr 82  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	4.011		4.352		0.341		4.011	0.050		3.683
FGB[10]:  	4.008		4.354		0.346		4.008	0.052		4.568
FGB[20]:  	4.025		4.356		0.331		4.025	0.054		4.362
XGB[5]:  	4.048		4.389		0.342 

prediciting using 83 trees
9146 9146
prediciting using 83 trees
computing last 1 trees of S [total trees: 84]
S Values before  [1.3327718322438928, 20.29274121257337, -20.62580502287435, -42.22789306486093, -3.2660721713872403, -1.6783503582110495, -0.6744346718064135, -

rmse before: 7.689419023981934
prediciting using 84 trees
prediciting using 84 trees
rmse after: 4.079236112049734
prediciting using 84 trees
prediciting using 84 trees
prediciting using 84 trees
2 Back
prediciting using 83 trees
9146 9146
prediciting using 64 trees
computing last 20 trees of S [total trees: 84]
S Values before  [-0.009999999967555127, 0.09999999958753607, -0.009999999946093054, -0.009999999547467232, 0.09999999965970807, 0.0999999999408268, 0.09999999996527342, -0.009999999957549062, -0.009999999544811844, -0.00999999994357239, -0.00999999995505853, 0.09000000000000001, -0.00999999996214213, 0.09999999995267145, -0.009999999576257555, 0.09999999948773941, 0.09999999996046348, -0.00999999959489114, -0.009999999952610843, 0.0999999995916912, -0.00999999995581531, -0.009999999926485766, 0.030097622669822983, 0.09999999999470231, 0.09999999950619883, 0.09999999957558146, -0.009999999580796637, 0.09999999996001642, -0.009999999956986468, -0.009999999422849666, 0.0999999999

rmse before: 4.067336809211044
prediciting using 84 trees
prediciting using 84 trees
rmse after: 4.090248372247523
prediciting using 84 trees
prediciting using 84 trees
prediciting using 84 trees
3 Back
[14:29:34] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 84 trees
prediciting using 84 trees
prediciting using 84 trees

-----------------------------------------------
ittr 83  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	4.003		4.346		0.343		4.003	0.053		2.071
FGB[10]:  	4.005		4.352		0.347		4.005	0.049		3.200
FGB[20]:  	4.015		4.348		0.333		4.015	0.049		2.608
XGB[5]:  	4.038		4.383		0.346 

prediciting using 84 trees
9146 9146
prediciting using 84 trees
computing last 1 trees of S [total trees: 85]
S Values before  [1.2052461032202773, -1.0262017252535662, -5.037360884574497, 1.759273641461388, 2.7230034081775103, -0.8785872009536335, -4.338978250716083, -0

rmse before: 4.047225587537107
prediciting using 85 trees
prediciting using 85 trees
rmse after: 4.073087703894935
prediciting using 85 trees
prediciting using 85 trees
prediciting using 85 trees
2 Back
prediciting using 84 trees
9146 9146
prediciting using 65 trees
computing last 20 trees of S [total trees: 85]
S Values before  [0.09999999980085528, -0.009999998558517829, -0.009999998811243187, 0.09999999826129531, 0.09999999999999999, -0.009999999824392331, -0.009999998950292136, -0.009999998876609122, 0.09000000000000001, 0.09000000000000001, -0.009999999771797032, 0.09000000000000001, -0.00999999908928052, 0.09999999986715953, -0.00999999896040938, -0.009999999748006588, 0.0999999999391701, 0.0999999998534335, -0.009999998736070773, 0.09999999887258114, 0.09999999981043535, 0.09999999990503379, 0.09999999868350365, 0.09999999971055837, -0.009999995387046842, 0.0417080874903517, 0.09999999812133169, 0.09999999793950189, 0.09999999985431114, 0.09999999912536994, 0.09999999988758541, 

rmse before: 4.057288738390215
prediciting using 85 trees
prediciting using 85 trees
rmse after: 4.087213325011856
prediciting using 85 trees
prediciting using 85 trees
prediciting using 85 trees
3 Back
[14:38:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGB
prediciting using 85 trees
prediciting using 85 trees
prediciting using 85 trees

-----------------------------------------------
ittr 84  	Train		Test		Diff		RVar	SVar	TVar
FGB[1]:  	3.998		4.343		0.345		3.998	0.054		1.956
FGB[10]:  	3.999		4.348		0.350		3.999	0.044		2.629
FGB[20]:  	4.012		4.346		0.334		4.012	0.055		3.282
XGB[5]:  	4.035		4.382		0.347 

prediciting using 85 trees
9146 9146
prediciting using 85 trees
computing last 1 trees of S [total trees: 86]
S Values before  [1.6098676296236396, 0.6719162622602757, 2.763161921657627, -0.11025277566620159, -474.88601456504375, 0.4290921376369398, 1.2204597770575998, 1

In [319]:
print(X)
print(minus(Y,mdl.predict(X,1)))
for x,y in zip(X,minus(Y,mdl.predict(X,1))):
    if x[0]<8:
        print(x,y)

[[1], [7], [8], [3], [4], [10], [6], [8], [5], [3]]
prediciting using 2 trees
[0.0, 0.0, 3.333333333333333, 1.0, 0.0, 1.333333333333333, 1.0, -4.666666666666667, 0.0, -2.0]
prediciting using 2 trees
[1] 0.0
[7] 0.0
[3] 1.0
[4] 0.0
[6] 1.0
[5] 0.0
[3] -2.0


In [193]:
import urllib# url of the data
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00265/CASP.csv'
# retrive data from the url and store in the folder name Data with filename as CASP.csv
urllib.request.urlretrieve (url, filename="CASP.csv" )

('CASP.csv', <http.client.HTTPMessage at 0x1b6360ea088>)

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split 
# Stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
df = pd.read_csv("CASP.csv")
labels = [1,2,3,4,5]
df['RMSD_cat'] = pd.cut(df['RMSD'], 5, labels=labels)
df.head()
# Create the instance from object
split = StratifiedShuffleSplit(n_splits=1, test_size=0.50, random_state=42)

# Create Train - Test Set
for train_index, test_index in split.split(df, df["RMSD_cat"]):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

# Removing the RMSD_cat to give the original data
for _set in (train_set, test_set):
    _set.drop(["RMSD_cat"], axis=1, inplace=True)
    
# Creating new copies of Stratified Train and Test data
y_train = train_set["RMSD"].copy().values.tolist()
X_train = train_set.drop("RMSD", axis=1).values.tolist()

y_test = test_set["RMSD"].copy().values.tolist()
X_test = test_set.drop("RMSD", axis=1).values.tolist()
print("Shape of X_train:", len(X_train),len(X_train[0]))
print("Shape of X_test:",len(X_test),len(X_test[0]))
savedf()
type(X_train)

Shape of X_train: 22865 9
Shape of X_test: 22865 9


list

In [ ]:
def removeXY(node):
    if node == None:
        return
    node.X = node.Y = ''
    removeXY(node.Left)
    removeXY(node.Right)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(500, 500))
removeXY(mdl.trees[85][0])
mdl.trees[85][0].draw_tree(ax=ax)
plt.show()

